# **Robust test matrix** 
- for each route that hits happy paths, edges, and failure cases
---

# Weather

## POST `/weather/current`

### ✅ Happy paths

1. **Free text**

```json
{ "query": "Hyderabad", "units": "metric" }
```

**Expect:** 200, `source=openweather`, sensible temps.

2. **Coordinates**

```json
{ "lat": 17.385, "lon": 78.4867, "units": "metric" }
```

**Expect:** 200.

3. **Zip (if your geocoder supports it)**

```json
{ "zip_code": "10001", "country_code": "US", "units": "imperial" }
```

**Expect:** 200.

### ❌ Error paths

4. **Missing location**

```json
{ "units": "metric" }
```

**Expect:** 400 "No location provided".

5. **Bad coords**

```json
{ "lat": 200, "lon": 0 }
```

**Expect:** 422 (validator: lat bounds).

6. **Unresolvable query**

```json
{ "query": "zzzz-not-a-real-place-12345" }
```

**Expect:** 404 "Location not found".

---

## POST `/weather/forecast`

### ✅ Happy paths

1.

```json
{ "query": "Hyderabad", "units": "metric", "days": 5 }
```

**Expect:** 200; `forecast` length 5.

2. **Min/max days**

```json
{ "query": "Hyderabad", "days": 1 }
```

```json
{ "query": "Hyderabad", "days": 5 }
```

**Expect:** 200.

### ❌ Error paths

3. **Missing location**

```json
{ "days": 3 }
```

**Expect:** 400.

4. **Days out of range**

```json
{ "query": "Hyderabad", "days": 6 }
```

**Expect:** 422 (Pydantic range check).

---

# Locations

## POST `/locations/search`

### ✅

```json
{ "query": "Charminar Hyderabad" }
```

**Expect:** 200; `candidates[0].lat/lon/formatted`.

### ❌

```json
{ "query": "x" }    // too short
```

**Expect:** 422 (min\_length=2).

---

# History (CRUD)

## POST `/weather/history`

### ✅ Happy paths

1. **Full record**

```json
{
  "location_name": "Hyderabad",
  "latitude": 17.385,
  "longitude": 78.4867,
  "start_date": "2025-08-15",
  "end_date": "2025-08-15",
  "weather_data": { "condition": "clear", "wind_speed": 3.2 },
  "temperature": 32.5,
  "humidity": 60.2,
  "recorded_at": "2025-08-15T14:30:00"
}
```

**Expect:** 200; new `id`, echo of fields.

2. **Minimal valid**

```json
{
  "location_name": "Pune",
  "temperature": 27.0,
  "humidity": 70.0,
  "recorded_at": "2025-08-16T10:00:00"
}
```

**Expect:** 200.

### ❌ Error paths

3. **Bad type**

```json
{
  "location_name": "Pune",
  "temperature": "thirty",
  "humidity": 70.0,
  "recorded_at": "2025-08-16T10:00:00"
}
```

**Expect:** 422.

4. **Missing required**

```json
{ "location_name": "Pune" }
```

**Expect:** 422 (needs temperature, humidity, recorded\_at).

---

## GET `/weather/history`

* `skip=0&limit=1` → **Expect:** at most 1 record.
* `skip=1&limit=1` → **Expect:** next record (pagination sanity).

---

## GET `/weather/history/search`

### ✅

* `location_name=Hydera` (partial, any case)
* `start_date=2025-08-01`
* `end_date=2025-08-31`
  **Expect:** matches for “Hyderabad” due to case-insensitive partial match.

### ❌

* `start_date=2025-08-31&end_date=2025-08-01`
  **Expect:** 400 "Start date cannot be after end date."

* `start_date=31-08-2025`
  **Expect:** 400 "Invalid date format."

---

## PUT `/weather/history/{id}`

### ✅ Partial update

```json
{ "temperature": 33.1, "humidity": 58.0, "weather_data": { "alert": "Heat advisory" } }
```

**Expect:** 200 updated object (other fields unchanged).

### ❌ Not found

`/weather/history/999999`

```json
{ "temperature": 30.0 }
```

**Expect:** 404.

---

## DELETE `/weather/history/{id}`

* **Existing id**: 200 returns deleted object.
* **Repeat delete** same id: 404.

---

# Export

> Remember: exports now use **case-insensitive partial** match for `location_name`.

## GET `/export/csv`

* `?location_name=hyd&start_date=2025-08-01&end_date=2025-08-31&limit=1000&skip=0`
  **Expect:** download with rows.
* `?location_name=notfound` → **Expect:** CSV with only header row (empty set).

## GET `/export/json`

* No filters → all rows (bounded by `limit`).
* `?limit=1` → single element array.

## GET `/export/xml`

* Bad date: `start_date=31-08-2025` → **Expect:** 400.

## GET `/export/pdf`

* If many rows, first 200 are included with an info note.

---

# YouTube

## GET `/youtube/{location}`

### ✅

`/youtube/Hyderabad?topic=travel&max_results=6`
**Expect:** 200, array of objects `{videoId,title,url,thumbnail,...}`

### ❌

* Missing key (`YOUTUBE_API_KEY` not set) → **Expect:** 400 `"YouTube API key not configured"`.
* Very large `max_results` → capped by schema at 25 (422 if >25).

---

# Maps

## GET `/maps/{location}`

### ✅

`/maps/Hyderabad?zoom=11&width=800&height=500`
**Expect:** 200 with `static_map_url`; open it in browser to verify.

### ❌

* Unknown place: `/maps/zzzz-not-a-real-place` → **Expect:** 404 "Location not found".
* Out-of-range params (e.g., width=150) → **Expect:** 422 (min 200).

---

# Chaos & Resilience (optional but solid for interviews)

* **Upstream failures (simulate):**

  * Temporarily set an invalid `OPENWEATHER_API_KEY` and hit `/weather/current` → **Expect:** 502 "Weather API failed: ..."
  * Temporarily remove `YOUTUBE_API_KEY` → **Expect:** 400.

* **Throughput limits:**

  * Call `/export/json?limit=50000` with many rows to ensure memory is fine (SQLite + Pandas should handle; PDF is limited to 200 rows by design).

* **Fuzzing inputs:**

  * `location_name` with SQL-ish strings: `"Hydera%' OR 1=1 --"` (should just behave like a string; SQLAlchemy params keep you safe).
  * Non-UTF8 characters in location/query → expect normal behavior; JSON export uses UTF-8.

---

# Quick curl pack (copy/paste)

```bash
# Weather Current (free text)
curl -s localhost:8000/weather/current -H "Content-Type: application/json" -d '{"query":"Hyderabad","units":"metric"}' | jq .

# Forecast (5 days)
curl -s localhost:8000/weather/forecast -H "Content-Type: application/json" -d '{"query":"Hyderabad","units":"metric","days":5}' | jq .

# Locations search
curl -s localhost:8000/locations/search -H "Content-Type: application/json" -d '{"query":"Charminar Hyderabad"}' | jq .

# History create
curl -s localhost:8000/weather/history -H "Content-Type: application/json" -d '{
  "location_name":"Hyderabad",
  "latitude":17.385,"longitude":78.4867,
  "start_date":"2025-08-15","end_date":"2025-08-15",
  "temperature":32.5,"humidity":60.2,
  "recorded_at":"2025-08-15T14:30:00",
  "weather_data":{"condition":"clear","wind_speed":3.2}
}' | jq .

# History search (partial, case-insensitive)
curl -s "localhost:8000/weather/history/search?location_name=hyd&start_date=2025-08-01&end_date=2025-08-31" | jq .

# Export CSV
curl -L "localhost:8000/export/csv?location_name=hyd&start_date=2025-08-01&end_date=2025-08-31" -o out.csv

# YouTube
curl -s "localhost:8000/youtube/Hyderabad?topic=travel&max_results=6" | jq .

# Maps
curl -s "localhost:8000/maps/Hyderabad?zoom=11&width=800&height=500" | jq .
```

